In [ ]:
import pandas as pd 

with open("./coords_xtc_dump.txt" , "r") as f:
    data = f.readlines()
strt_idxs = [ i for i, d in enumerate(data)  if "coords.xtc" in d] 
trjs = []
for i, strt in enumerate(strt_idxs):
    try: 
        trjs.append(data[strt_idxs[i] : strt_idxs[i+1]])
    except: 
        trjs.append(data[strt_idxs[i] : ])
boxes = []
coords = []
for trj in trjs:
    box = trj[3:6]
    coord = trj[7:]
    newcoord = []   
    for i in coord:
        xyz = [ float(x.strip()) for x in i.split("{")[1].split("}")[0].split(",")]
        newcoord.append(xyz)
    coords.append(newcoord)

    newbox = []
    for i in box: 
        boxvec = [ float(x.strip()) for x in i.split("{")[1].split("}")[0].split(",")]
        newbox.append(boxvec)
    boxes.append(newbox)


In [72]:
import numpy as np
'''coords[0]은 40 x 3 리스트. 이때 0,1 ,12번 인덱스 위치를 가지고 평면벡터를 만들거야 그리고 그 평면벡터를 xy 평면으로 놓이게끔할거고
이떄 12번 원자->1번원자 벡터가 x축과평행하며 12번원자는 원점에 놓일거야
전체 시스템을 돌리고 평행이동시켜서 align시키는 코드 '''
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import pandas as pd 
def rotation_matrix_from_vectors(a, b):
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    v = np.cross(a, b)
    c = np.dot(a, b)
    if np.isclose(c, -1.0):
        return -np.eye(3)  # 180도 회전
    if np.isclose(c, 1.0):
        return np.eye(3)  # 동일
    vx = np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])
    R = np.eye(3) + vx + vx @ vx * ((1 - c) / (np.linalg.norm(v) ** 2))
    return R
def align_frame(frame_coords):
    coords = np.array(frame_coords)  # (40, 3)

    # 기준점 추출
    p0 = coords[0]
    p1 = coords[1]
    p12 = coords[12]

    # 평면 법선벡터 (0-12-1)
    v1 = p0 - p12
    v2 = p1 - p12
    normal = np.cross(v1, v2)
    normal /= np.linalg.norm(normal)

    # 1단계 회전: 평면 normal → z축
    R1 = rotation_matrix_from_vectors(normal, np.array([0, 0, 1]))
    coords_rot1 = coords @ R1.T

    # 2단계 회전: xy 평면에 투영된 (0-12) → x축
    new_p0 = coords_rot1[0]
    new_p12 = coords_rot1[12]
    vec = new_p0 - new_p12
    vec_xy = np.array([vec[0], vec[1], 0.0])
    if np.linalg.norm(vec_xy) < 1e-8:
        raise ValueError("12→0 벡터의 xy 성분이 거의 0입니다. 회전 정의가 불가능합니다.")
    vec_xy /= np.linalg.norm(vec_xy)
    R2 = rotation_matrix_from_vectors(vec_xy, np.array([1, 0, 0]))
    coords_rot2 = coords_rot1 @ R2.T

    # 평행이동
    coords_aligned = coords_rot2 - coords_rot2[12]

    # ✅ 리스케일: 원본과 정렬된 좌표계에서의 기준 거리 비교
    d_original = np.linalg.norm(p0 - p12)
    d_aligned = np.linalg.norm(coords_aligned[0] - coords_aligned[12])
    scale_factor = d_original / d_aligned
    coords_scaled = coords_aligned * scale_factor

    return coords_scaled, coords, [p0, p1, p12], normal
def plot_before_after_mpl(original, aligned, triangle_pts, normal_vec):
    fig = plt.figure(figsize=(14, 6))
    tri = np.array(triangle_pts)
    center = np.mean(tri, axis=0)
    axis_len = 1.5

    # ▶️ 원본 좌표계
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.set_title("Before Alignment")
    original = np.array(original)

    # 음이온 (0~14), 양이온 (15~)
    ax1.scatter(*original[:15].T, color='#1f77b4', s=20, label='Anion')
    ax1.scatter(*original[15:].T, color='#ff7f0e', s=20, label='Cation')

    # 삼각형 평면 영역 색칠
    ax1.plot_trisurf(tri[:, 0], tri[:, 1], tri[:, 2], color='cyan', alpha=0.4)

    # 법선 벡터 (빨간색)
    ax1.quiver(*center, *normal_vec, length=1.0, color='red', linewidth=2)

    ax1.legend()

    # ▶️ 정렬된 좌표계
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.set_title("After Alignment")
    aligned = np.array(aligned)

    # 음이온 / 양이온 구분 시각화
    ax2.scatter(*aligned[:15].T, color='#1f77b4', s=20, label='Anion')
    ax2.scatter(*aligned[15:].T, color='#ff7f0e', s=20, label='Cation')

    # 정렬된 평면 및 법선벡터 다시 계산
    tri_aligned = aligned[[0, 1, 12]]
    center_aligned = np.mean(tri_aligned, axis=0)
    v1 = tri_aligned[0] - tri_aligned[2]
    v2 = tri_aligned[1] - tri_aligned[2]
    normal_aligned = np.cross(v1, v2)
    normal_aligned /= np.linalg.norm(normal_aligned)

    # 삼각형 평면 영역 색칠
    ax2.plot_trisurf(tri_aligned[:, 0], tri_aligned[:, 1], tri_aligned[:, 2], color='cyan', alpha=0.4)

    # xyz 축 (검정)
    ax2.plot([0, axis_len], [0, 0], [0, 0], 'k')  # x
    ax2.plot([0, 0], [0, axis_len], [0, 0], 'k')  # y
    ax2.plot([0, 0], [0, 0], [0, axis_len], 'k')  # z

    # 법선 벡터 (빨간색)
    ax2.quiver(*center_aligned, *normal_aligned, length=1.0, color='red', linewidth=2)

    ax2.legend()
    plt.tight_layout()
    plt.show()
def plot_before_after_plotly(original, aligned, triangle_pts, normal_vec, html = False):
    original = np.array(original)
    aligned = np.array(aligned)
    tri = np.array(triangle_pts)
    center = np.mean(tri, axis=0)

    ## ---------- 공통 요소 ----------
    def mesh_triangle(tri_coords):
        return go.Mesh3d(
            x=tri_coords[:, 0], y=tri_coords[:, 1], z=tri_coords[:, 2],
            color='cyan', opacity=0.4,
            i=[0], j=[1], k=[2],
            name='Triangle'
        )

    def normal_arrow_line(center, vector, length=1.0, opacity=0.5):
        end = center + vector / np.linalg.norm(vector) * length
        return go.Scatter3d(
            x=[center[0], end[0]],
            y=[center[1], end[1]],
            z=[center[2], end[2]],
            mode='lines',
            line=dict(color='red', width=6),
            opacity=opacity,
            name='Normal'
    )

    def axis_lines():
        axis_len = 1.5
        return [
            go.Scatter3d(x=[0, axis_len], y=[0, 0], z=[0, 0], mode='lines', line=dict(color='black'), showlegend=False),
            go.Scatter3d(x=[0, 0], y=[0, axis_len], z=[0, 0], mode='lines', line=dict(color='black'), showlegend=False),
            go.Scatter3d(x=[0, 0], y=[0, 0], z=[0, axis_len], mode='lines', line=dict(color='black'), showlegend=False)
        ]

    ## ---------- Before ----------
    trace_before_anion = go.Scatter3d(
        x=original[:15, 0], y=original[:15, 1], z=original[:15, 2],
        mode='markers',
        marker=dict(color='#1f77b4', size=4),
        name='Anion (Before)'
    )

    trace_before_cation = go.Scatter3d(
        x=original[15:, 0], y=original[15:, 1], z=original[15:, 2],
        mode='markers',
        marker=dict(color='#ff7f0e', size=4),
        name='Cation (Before)'
    )

    trace_before_triangle = mesh_triangle(tri)
    # Before scene
    trace_before_normal = normal_arrow_line(center, normal_vec, length=1.0, opacity=0.5)

        # After scene


    ## ---------- After ----------
    tri_aligned = aligned[[0, 1, 12]]
    center_aligned = np.mean(tri_aligned, axis=0)
    v1 = tri_aligned[0] - tri_aligned[2]
    v2 = tri_aligned[1] - tri_aligned[2]
    normal_aligned = np.cross(v1, v2)
    normal_aligned /= np.linalg.norm(normal_aligned)

    trace_after_anion = go.Scatter3d(
        x=aligned[:15, 0], y=aligned[:15, 1], z=aligned[:15, 2],
        mode='markers',
        marker=dict(color='#1f77b4', size=4),
        name='Anion (After)'
    )

    trace_after_cation = go.Scatter3d(
        x=aligned[15:, 0], y=aligned[15:, 1], z=aligned[15:, 2],
        mode='markers',
        marker=dict(color='#ff7f0e', size=4),
        name='Cation (After)'
    )

    trace_after_triangle = mesh_triangle(tri_aligned)
    trace_after_normal = normal_arrow_line(center_aligned, normal_aligned, length=1.0, opacity=0.5)
    trace_after_axes = axis_lines()

    ## ---------- Layout ----------
    fig = go.Figure()

    # 좌: Before
    for trace in [trace_before_anion, trace_before_cation, trace_before_triangle, trace_before_normal]:
        trace.update(scene='scene')
        fig.add_trace(trace)

    # 우: After
    for trace in [trace_after_anion, trace_after_cation, trace_after_triangle, trace_after_normal] + trace_after_axes:
        trace.update(scene='scene2')
        fig.add_trace(trace)

    fig.update_layout(
        scene=dict(
            domain=dict(x=[0, 0.48]),  # 왼쪽 그래프 여백 확보
            xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
            aspectmode='data',
        ),
        scene2=dict(
            domain=dict(x=[0.52, 1.0]),  # 오른쪽 그래프 여백 확보
            xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
            aspectmode='data',
        ),
        margin=dict(l=0, r=0, t=30, b=0),
        height=600,
        title="Before vs After Alignment (Interactive)"
    )



    if html:
        fig.write_html("alignment_comparison.html")
    fig.show()

with open("./coords_xtc_dump.txt" , "r") as f:
    data = f.readlines()
strt_idxs = [ i for i, d in enumerate(data)  if "coords.xtc" in d] 
trjs = []
for i, strt in enumerate(strt_idxs):
    try: 
        trjs.append(data[strt_idxs[i] : strt_idxs[i+1]])
    except: 
        trjs.append(data[strt_idxs[i] : ])
boxes = []
coords = []
for trj in trjs:
    box = trj[3:6]
    coord = trj[7:]
    newcoord = []   
    for i in coord:
        xyz = [ float(x.strip()) for x in i.split("{")[1].split("}")[0].split(",")]
        newcoord.append(xyz)
    coords.append(newcoord)

    newbox = []
    for i in box: 
        boxvec = [ float(x.strip()) for x in i.split("{")[1].split("}")[0].split(",")]
        newbox.append(boxvec)
    boxes.append(newbox)

# 예시 좌표 데이터로 실행
aligned, original, triangle_pts, normal_vec = align_frame(coords[-1])
# 시각화 실행
# plot_before_after_mpl(original, aligned, triangle_pts, normal_vec)
plot_before_after_plotly(original, aligned, triangle_pts, normal_vec)

aligns = []
atoms = ["S","S","F","F","F","F","F","F","O","O","O","O","N","C","C","N","N","C","C","C","C","C","C","C","C","H","H","H","H","H","H","H","H","H","H","H","H","H","H","H" ]
xyzfiles = []
for c in coords: 
    xyzfile = f'''{len(atoms)}\n\n'''
    aligned, original, triangle_pts, normal_vec = align_frame(c)
    aligns.append(aligned)
    for atom, line in zip ( atoms, aligned): 
        xyz_ = ' '.join([f"{x*10:.6f}" for x in line])
        xyzfile+=f"{atom} {xyz_}\n"
    xyzfiles.append(xyzfile)
# print(xyzfile)
integrated_xyz = ''.join(xyzfiles)
with open( "integrated_xyz.xyz" , 'w') as f:
    f.write(integrated_xyz)

In [ ]:
with open("./integrated_xyz.xyz", 'r')  as f:
    data =f.read().split("\n\n")[-1]
data

'S 5.307348 0.952179 2.930735\nS 2.519657 1.267782 3.507429\nF 6.461616 0.185239 5.088713\nF 4.900412 -1.112827 4.370684\nF 6.868050 -1.081607 3.367227\nF 0.526510 0.948522 5.249047\nF 2.638943 0.499446 6.046076\nF 1.867179 2.573157 5.745903\nO 6.433663 1.855071 2.842409\nO 4.810588 0.296986 1.752410\nO 2.333040 -0.156612 3.087814\nO 1.885503 2.297160 2.716714\nN 4.111689 1.693910 3.705291\nC 5.918795 -0.346043 3.987358\nC 1.851442 1.331326 5.286656\nN 1.391618 0.000000 -0.000000\nN 0.744026 2.131234 0.025822\nC 2.203694 -1.235933 -0.168232\nC 2.560976 -1.645741 -1.633260\nC 3.363639 -2.941645 -1.893767\nC 1.800063 1.336552 0.000000\nC 0.000000 0.000000 -0.000000\nC -0.404565 1.332864 -0.047960\nC 2.645338 -4.335989 -1.626279\nC 0.759045 3.638744 0.055634\nH 1.684514 -2.008625 0.399923\nH 3.158770 -1.098936 0.359464\nH 1.558618 -1.903263 -1.996522\nH 3.003986 -0.916960 -2.330913\nH 4.273515 -2.873736 -1.301388\nH 3.815487 -2.925900 -2.890755\nH 2.820602 1.669323 0.134774\nH -0.569769 -

In [67]:
print(xyzfiles[0])

40

S 0.157258 0.000000 0.000000
S -0.079945 0.142477 0.000000
F 0.337535 -0.055726 -0.200551
F 0.164342 0.083033 -0.254699
F 0.128541 -0.129605 -0.242565
F -0.298720 -0.003483 -0.008329
F -0.237470 0.094081 -0.200474
F -0.328111 0.209304 -0.035984
O 0.213783 0.121801 0.045104
O 0.200683 -0.123011 0.063438
O -0.105546 0.179994 0.140353
O -0.018893 0.231873 -0.097852
N 0.000000 0.000000 0.000000
C 0.203627 -0.028615 -0.187294
C -0.246195 0.108073 -0.067383
N 0.096676 -0.120765 0.342512
N 0.162478 0.083953 0.391191
C 0.014638 -0.235137 0.290341
C -0.089604 -0.281255 0.401407
C -0.220091 -0.192551 0.417249
C 0.052233 0.010709 0.360415
C 0.233432 -0.118989 0.362922
C 0.276082 0.004684 0.397257
C -0.333275 -0.262931 0.497730
C 0.166040 0.234845 0.402806
H -0.022314 -0.211766 0.189355
H 0.084117 -0.319042 0.276495
H -0.032399 -0.294216 0.493708
H -0.135529 -0.378656 0.382173
H -0.194388 -0.102201 0.472641
H -0.273704 -0.186151 0.321778
H -0.035979 0.038846 0.303459
H 0.286536 -0.212384 0.352

'0.160614 -0.000000 0.000000'